## Load

In [1]:
import json

In [2]:
with open("raw/ltl.txt", "r") as f:
    raw_ltls = f.read().splitlines()
    unique_ltls = set(raw_ltls)
with open("raw/eng.txt", 'r') as f:
    raw_engs = f.read().splitlines()

DPs = []
for ltl, eng in zip(raw_ltls, raw_engs):
    DPs.append({'ltl': ltl, 'eng': eng})

In [3]:
for ltl in sorted(unique_ltls, key=lambda x: len(x)):
    print(ltl)

F ( red_room )
F ( blue_room )
F ( landmark_2 )
F ( landmark_1 )
F ( landmark_3 )
F ( green_room )
F ( purple_room )
F ( orange_room )
F ( third_floor )
F ( yellow_room )
F ( first_floor )
F ( second_floor )
F ( red_room & ~ blue_room )
F ( red_room & ~ landmark_3 )
F ( red_room & ~ landmark_1 )
F ( landmark_3 & ~ red_room )
F ( landmark_1 & ~ red_room )
F ( green_room & ~ red_room )
F ( landmark_2 & ~ red_room )
F ( red_room & ~ green_room )
F ( red_room & ~ landmark_2 )
F ( blue_room & ~ landmark_1 )
F ( landmark_2 & ~ blue_room )
F ( red_room & ~ yellow_room )
F ( orange_room & ~ red_room )
F ( purple_room & ~ red_room )
F ( third_floor & ~ red_room )
F ( first_floor & ~ red_room )
F ( landmark_3 & ~ blue_room )
F ( blue_room & ~ landmark_3 )
F ( red_room & ~ orange_room )
F ( green_room & ~ blue_room )
F ( landmark_1 & ~ blue_room )
F ( yellow_room & ~ red_room )
~ ( red_room ) U ( blue_room )
F ( red_room & ~ purple_room )
F ( blue_room & ~ landmark_2 )
F ( landmark_2 & ~ green_ro

In [4]:
print("Drone-planning Domain, with augmentation")
print("Number of Data Points", len(DPs))
print("Number of unique LTL expressions:", len(unique_ltls))
print("Number of unique LTL structures:", 5)

Drone-planning Domain, with augmentation
Number of Data Points 6185
Number of unique LTL expressions: 355
Number of unique LTL structures: 5


F ( AP )
- Go to AP

F ( AP1 & F ( AP2 ) )
- First go to AP1 and then to AP2


F ( AP1 ) & G ( AP2 )
- Go to AP1 and Always in AP2


F ( AP1 ) & ~ ( AP2 )
- Go to AP1 and avoid AP2 

~ ( AP1 ) U ( AP2 )
- avoid going through AP1 until go through AP2


## Clean up LTL expressions 

In [5]:
ori_aps = set()
for unique_ltl in unique_ltls:
    ori_aps.update(unique_ltl.split())

In [6]:
ori_aps

{'&',
 '(',
 ')',
 'F',
 'G',
 'U',
 'blue_room',
 'first_floor',
 'green_room',
 'landmark_1',
 'landmark_2',
 'landmark_3',
 'orange_room',
 'purple_room',
 'red_room',
 'second_floor',
 'third_floor',
 'yellow_room',
 '~'}

In [7]:
APs = [
    {'ap': 'blue_room', 'eng': 'the blue room'},
    {'ap': 'green_room', 'eng': 'the green room'},
    {'ap': 'orange_room', 'eng': 'the orange room'},
    {'ap': 'purple_room', 'eng': 'the purple room'},
    {'ap': 'red_room', 'eng': 'the red room'},
    {'ap': 'yellow_room', 'eng': 'the yellow room'},
    {'ap': 'first_floor', 'eng': 'the first floor'},
    {'ap': 'second_floor', 'eng': 'the second floor'},
    {'ap': 'third_floor', 'eng': 'the third floor'},
    {'ap': 'landmark_1', 'eng': 'landmark 1'},
    {'ap': 'landmark_2', 'eng': 'landmark 2'},
    {'ap': 'landmark_3', 'eng': 'landmark 3'},
]

In [9]:
# from the translator
def build_type_A(ap1):
    raw_ltl = f"F ( {ap1['ap']} )"
    raw_ltl = f"F ( {ap1['ap']} )"
    canonical_ltl = f"finally ( {ap1['eng']} )"
    translated = f"go to {ap1['eng']}"
    return {
        'raw_ltl': raw_ltl,
        'canonical_ltl': canonical_ltl,
        'eng': translated}

def build_type_B(room_1, room_2):
    raw_ltl = f"F ( {room_1['ap']} & F ( {room_2['ap']} ) )"
    canonical_ltl = f"finally ( and ( {room_1['eng']} , finally ( {room_2['eng']} ) ) )"
    translated = f"go to {room_1['eng']}, and then {room_2['eng']} at last"
    return {
        'raw_ltl': raw_ltl,
        'canonical_ltl': canonical_ltl,
        'eng': translated}

def build_type_C(room_1, room_2):
    raw_ltl = f"F ( {room_1['ap']} ) & G ( {room_2['ap']} )"
    canonical_ltl = f"and ( finally ( {room_1['eng']} ) , always ( {room_2['eng']} ) )"
    translated = f"go to {room_1['eng']} at last, and always stay in {room_2['eng']}"
    return {
        'raw_ltl': raw_ltl,
        'canonical_ltl': canonical_ltl,
        'eng': translated}

def build_type_D(room_1, room_2):
    raw_ltl = f"F ( {room_1['ap']} & ~ {room_2['ap']} )"
    canonical_ltl = f"finally ( and ( {room_1['eng']} , not ( {room_2['eng']} ) ) )"
    translated = f"go to {room_1['eng']} at last, and avoid {room_2['eng']}"
    return {
        'raw_ltl': raw_ltl,
        'canonical_ltl': canonical_ltl,
        'eng': translated}

def build_type_E(room_1, room_2):
    raw_ltl = f"~ ( {room_1['ap']} ) U ( {room_2['ap']} )"
    canonical_ltl = f"until ( not ( {room_1['eng']} ) , {room_2['eng']} )"
    translated = f"avoid go to {room_1['eng']} until go to {room_2['eng']}"
    return {
        'raw_ltl': raw_ltl,
        'canonical_ltl': canonical_ltl,
        'eng': translated}

In [10]:
translation_seeds = []
for r1 in APs:
    translation_seeds.append(build_type_A(r1))
    for r2 in APs:
        if r1 == r2:
            continue
        translation_seeds.append(build_type_B(r1, r2))
        translation_seeds.append(build_type_C(r1, r2))
        translation_seeds.append(build_type_D(r1, r2))
        translation_seeds.append(build_type_E(r1, r2))

In [11]:
print(len(translation_seeds))

540


In [12]:
seed_ltls = set([t['raw_ltl'] for t in translation_seeds])
print(unique_ltls - seed_ltls)

set()


### Save the canonical decoding list

In [13]:
possible_decodings = {}
for seed in translation_seeds:
    canonical = seed['canonical_ltl']
    possible_decodings[canonical] = {
        'formula': canonical,
        'raw': seed['raw_ltl'],
    }

In [14]:
with open("canonical.json", 'w') as f:
    f.write(json.dumps(possible_decodings, indent=2))

### Save translation seed for zero shot

In [15]:
with open("train_seed.jsonl", "w") as f:
    for dp in translation_seeds:
        better_ltl = dp['canonical_ltl']
        entry = {'canonical': better_ltl, 'formula': better_ltl, 'natural': dp['eng']}
        json.dump(entry, f)
        f.write('\n')

### Save golden data for evaluation  

In [16]:
raw_canonical_mapping = {
    seed['raw_ltl']: seed['canonical_ltl'] for seed in translation_seeds
}

In [17]:
len(DPs)

6185

In [18]:
DPs[0]

{'ltl': '~ ( purple_room ) U ( third_floor )',
 'eng': 'go to the twond floor avoiding the purple room . if you reached the third floor you can stay there , or head to the purple room .'}

In [19]:
with open("golden.jsonl", "w") as f:
    for dp in DPs:
        entry = {
            'canonical': raw_canonical_mapping[dp['ltl']],
            'formula': raw_canonical_mapping[dp['ltl']],
            'natural': dp['eng'],
            'raw_ltl': dp['ltl'],
        }
        json.dump(entry, f)
        f.write('\n')